In [1]:
from pyspark.context import SparkContext
from pyspark.sql.types import StringType
from pyspark.sql import Window, SparkSession
from pyspark.sql.functions import when, to_date, unix_timestamp, udf, row_number, concat, col, lit, concat_ws, monotonically_increasing_id,input_file_name, regexp_replace, length, expr
from datetime import datetime
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
#### connect to database and load the data
spark = SparkSession.builder.appName('Daliy_Data').getOrCreate()
spark.sparkContext.setLogLevel('WARN') ## ignore the warn

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/21 10:50:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
timelist = ['2023-06-24','2023-06-25','2023-06-26','2023-06-27','2023-06-28','2023-06-29','2023-06-30',\
            '2023-07-01','2023-07-02','2023-07-03','2023-07-04','2023-07-05','2023-07-06','2023-07-07','2023-07-08','2023-07-09',\
            '2023-07-10','2023-07-11','2023-07-12','2023-07-13','2023-07-14','2023-07-15','2023-07-16','2023-07-17','2023-07-18','2023-07-19']
platform_list = ['CH','WC','WA']
for i in range(26):
    tmp_time = timelist[i]
    ## get all price from 3 platform
    for j in range(3):
        tmp_pl = platform_list[j]
        file = 'data_'+tmp_pl+'_'+tmp_time[6]+tmp_time[8]+tmp_time[9]+'.csv'
        tmp_file = pd.read_csv('/Users/yudi/Desktop/wristcheck/price index/raw data/'+file)
        tmp_file = tmp_file.dropna(subset = ['price','reference_match'])
        if j == 0:
            tmp_s = tmp_file.copy()
        else:
            tmp_s = pd.concat([tmp_s,tmp_file])
    if i == 0:
        tmp_a = tmp_s.copy()
    else:
        tmp_a = pd.concat([tmp_a,tmp_s])

In [11]:
tmp_a.to_csv("/Users/yudi/Desktop/wristcheck/price index/recent_all.csv")

In [22]:
spark.createDataFrame(tmp_a).write.format("jdbc")\
     .option("url", "jdbc:postgresql://glue-demo-intern.cjlwasvkzi50.ap-southeast-1.rds.amazonaws.com:5432/wc_data") \
     .option("driver", "org.postgresql.Driver")\
     .option("dbtable", "wc_raw_after_623") \
     .option("user", "postgres")\
     .option("password", "B&DNXSmAQe9cTJ3S").save()

23/07/21 11:36:30 WARN TaskSetManager: Stage 0 contains a task of very large size (19599 KiB). The maximum recommended task size is 1000 KiB.
